In [1]:
import pickle
import random
import numpy
import torch
import os

from Auxiliary_Functions.generate_observation import generate_observations, split_dataset
from Auxiliary_Functions.save_progress import save_progress
from Auxiliary_Functions.matrix_normalization import normalize_max_row_norm

from Denoising_Algorithms.Projectors.DnCNN import DnCNN
from Denoising_Algorithms.DL_Training.evaluation import average_PSNR
from Denoising_Algorithms.DL_Training.loss_functions import IntermediateScaledLoss
from Denoising_Algorithms.DL_Training.training import train_main
from Denoising_Algorithms.Memory_Network.Memory_Net import MemoryNetwork
from Denoising_Algorithms.DL_Training.evaluation import average_PSNR

In [2]:
current_trial = 0

torch.manual_seed(current_trial)
numpy.random.seed(current_trial)
random.seed(current_trial)
torch.cuda.manual_seed(current_trial)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [3]:
with open('Auxiliary_Functions/sensing_matrices/50 x 50 images/DCT_10.pkl', 'rb') as f:
    DCT_10 = pickle.load(f)
with open('Auxiliary_Functions/sensing_matrices/50 x 50 images/DCT_20.pkl', 'rb') as f:
    DCT_20 = pickle.load(f)
with open('Auxiliary_Functions/sensing_matrices/50 x 50 images/DCT_30.pkl', 'rb') as f:
    DCT_30 = pickle.load(f)
with open('Auxiliary_Functions/sensing_matrices/50 x 50 images/DCT_40.pkl', 'rb') as f:
    DCT_40 = pickle.load(f)

In [4]:
with open('ILSVRC2012_50x50_small.pkl', 'rb') as f:
    ILSVRC2012 = pickle.load(f)

# We only work with the first 25000 images
img_dataset = ILSVRC2012[:25000]

In [5]:
DCT_10_normalized = normalize_max_row_norm(DCT_10)
DCT_20_normalized = normalize_max_row_norm(DCT_20)
DCT_30_normalized = normalize_max_row_norm(DCT_30)
DCT_40_normalized = normalize_max_row_norm(DCT_40)

DCT_10_observations = generate_observations(img_dataset, DCT_10_normalized, 0.01)
DCT_20_observations = generate_observations(img_dataset, DCT_20_normalized, 0.01)
DCT_30_observations = generate_observations(img_dataset, DCT_30_normalized, 0.01)
DCT_40_observations = generate_observations(img_dataset, DCT_40_normalized, 0.01)

DCT_10_train, DCT_10_test = split_dataset(DCT_10_observations, train_ratio = 0.9, seed = 0)
DCT_20_train, DCT_20_test = split_dataset(DCT_20_observations, train_ratio = 0.9, seed = 0)
DCT_30_train, DCT_30_test = split_dataset(DCT_30_observations, train_ratio = 0.9, seed = 0)
DCT_40_train, DCT_40_test = split_dataset(DCT_40_observations, train_ratio = 0.9, seed = 0)

In [6]:
projector = DnCNN(depth = 3)

# Equal Weighting
loss_function = IntermediateScaledLoss(omega = 1.0)

# Device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [7]:
# 5 Projections
num_projections = 5

# Folder Path for 5 Projections
proj5_path = 'Memory_Residual-DCT-Additive_Depth3-.01/5 Projections'

DCT_10_proj5_weights, DCT_10_proj5_hist = train_main(DCT_10_normalized, [1] * num_projections, DCT_10_train, model=projector, device=device, model_type='Memory', split=0.2, 
                                                     numProjections=num_projections, residual=True, loss_function=loss_function, model_dir='Model_Paths/DCT_10_proj5', display=True)

DCT_20_proj5_weights, DCT_20_proj5_hist = train_main(DCT_20_normalized, [1] * num_projections, DCT_20_train, model=projector, device=device, model_type='Memory', split=0.2, 
                                                     numProjections=num_projections, residual=True, loss_function=loss_function, model_dir='Model_Paths/DCT_20_proj5', display=True)

DCT_30_proj5_weights, DCT_30_proj5_hist = train_main(DCT_30_normalized, [1] * num_projections, DCT_30_train, model=projector, device=device, model_type='Memory', split=0.2, 
                                                     numProjections=num_projections, residual=True, loss_function=loss_function, model_dir='Model_Paths/DCT_30_proj5', display=True)

DCT_40_proj5_weights, DCT_40_proj5_hist = train_main(DCT_40_normalized, [1] * num_projections, DCT_40_train, model=projector, device=device, model_type='Memory', split=0.2, 
                                                     numProjections=num_projections, residual=True, loss_function=loss_function, model_dir='Model_Paths/DCT_40_proj5', display=True)

# Store Best Model Weights and History

save_progress(proj5_path, DCT_10_proj5_weights, DCT_10_proj5_hist, '10% Sampling')
save_progress(proj5_path, DCT_20_proj5_weights, DCT_20_proj5_hist, '20% Sampling')
save_progress(proj5_path, DCT_30_proj5_weights, DCT_30_proj5_hist, '30% Sampling')
save_progress(proj5_path, DCT_40_proj5_weights, DCT_40_proj5_hist, '40% Sampling')

Validation Size: 4500, Train Size: 18000
Epoch 1/300
Average training loss: 0.060384024833639464
Average test loss: 0.005168065807885594
Epoch 2/300
Average training loss: 0.02371134506000413
Average test loss: 0.004640367968628804
Epoch 3/300
Average training loss: 0.022627102457814748
Average test loss: 0.0044687315614687076
Epoch 4/300
Average training loss: 0.02214720558292336
Average test loss: 0.0044698291255368125
Epoch 5/300
Average training loss: 0.021872589990496634
Average test loss: 0.00449177739272515
Epoch 6/300
Average training loss: 0.02166958801779482
Average test loss: 0.004340135991987255
Epoch 7/300
Average training loss: 0.02152130233993133
Average test loss: 0.004321808335267835
Epoch 8/300
Average training loss: 0.02140096071859201
Average test loss: 0.004306664714796676
Epoch 9/300
Average training loss: 0.021299885011381574
Average test loss: 0.004270886572698752
Epoch 10/300
Average training loss: 0.021212717754973305
Average test loss: 0.004289746989599532
Ep

In [8]:
DCT_10_proj5_model = MemoryNetwork(DCT_10_normalized, [1] * num_projections, num_projections, projector, device, residual=True).to(device)
DCT_20_proj5_model = MemoryNetwork(DCT_20_normalized, [1] * num_projections, num_projections, projector, device, residual=True).to(device)
DCT_30_proj5_model = MemoryNetwork(DCT_30_normalized, [1] * num_projections, num_projections, projector, device, residual=True).to(device)
DCT_40_proj5_model = MemoryNetwork(DCT_40_normalized, [1] * num_projections, num_projections, projector, device, residual=True).to(device)

DCT_10_proj5_psnr = average_PSNR(DCT_10_proj5_model, DCT_10_proj5_weights, num_projections, DCT_10_test, print_psnr=True)
DCT_20_proj5_psnr = average_PSNR(DCT_20_proj5_model, DCT_20_proj5_weights, num_projections, DCT_20_test, print_psnr=True)
DCT_30_proj5_psnr = average_PSNR(DCT_30_proj5_model, DCT_30_proj5_weights, num_projections, DCT_30_test, print_psnr=True)
DCT_40_proj5_psnr = average_PSNR(DCT_40_proj5_model, DCT_40_proj5_weights, num_projections, DCT_40_test, print_psnr=True)

with open(os.path.join(proj5_path, '10% Sampling', 'test_PSNR.pkl'), 'wb') as f:
    pickle.dump(DCT_10_proj5_psnr, f)
with open(os.path.join(proj5_path, '20% Sampling', 'test_PSNR.pkl'), 'wb') as f:
    pickle.dump(DCT_20_proj5_psnr, f)
with open(os.path.join(proj5_path, '30% Sampling', 'test_PSNR.pkl'), 'wb') as f:
    pickle.dump(DCT_30_proj5_psnr, f)
with open(os.path.join(proj5_path, '40% Sampling', 'test_PSNR.pkl'), 'wb') as f:
    pickle.dump(DCT_40_proj5_psnr, f)

Average PSNR for Projection Layer 0 across 2500 images: 27.58
Average PSNR for Projection Layer 1 across 2500 images: 27.84
Average PSNR for Projection Layer 2 across 2500 images: 28.06
Average PSNR for Projection Layer 3 across 2500 images: 28.19
Average PSNR for Projection Layer 4 across 2500 images: 28.22
Average PSNR for Projection Layer 0 across 2500 images: 28.20
Average PSNR for Projection Layer 1 across 2500 images: 28.78
Average PSNR for Projection Layer 2 across 2500 images: 29.19
Average PSNR for Projection Layer 3 across 2500 images: 29.44
Average PSNR for Projection Layer 4 across 2500 images: 29.66
Average PSNR for Projection Layer 0 across 2500 images: 28.90
Average PSNR for Projection Layer 1 across 2500 images: 29.88
Average PSNR for Projection Layer 2 across 2500 images: 30.34
Average PSNR for Projection Layer 3 across 2500 images: 30.74
Average PSNR for Projection Layer 4 across 2500 images: 31.03
Average PSNR for Projection Layer 0 across 2500 images: 29.70
Average 

In [9]:
# 15 Projections
num_projections = 15

# Folder Path for 15 Projections
proj15_path = 'Memory_Residual-DCT-Additive_Depth3-.01/15 Projections'

DCT_10_proj15_weights, DCT_10_proj15_hist = train_main(DCT_10_normalized, [1] * num_projections, DCT_10_train, model=projector, device=device, model_type='Memory', split=0.2, 
                                                     numProjections=num_projections, residual=True, loss_function=loss_function, model_dir='Model_Paths/DCT_10_proj15', display=True)

DCT_20_proj15_weights, DCT_20_proj15_hist = train_main(DCT_20_normalized, [1] * num_projections, DCT_20_train, model=projector, device=device, model_type='Memory', split=0.2, 
                                                     numProjections=num_projections, residual=True, loss_function=loss_function, model_dir='Model_Paths/DCT_20_proj15', display=True)

DCT_30_proj15_weights, DCT_30_proj15_hist = train_main(DCT_30_normalized, [1] * num_projections, DCT_30_train, model=projector, device=device, model_type='Memory', split=0.2, 
                                                     numProjections=num_projections, residual=True, loss_function=loss_function, model_dir='Model_Paths/DCT_30_proj15', display=True)

DCT_40_proj15_weights, DCT_40_proj15_hist = train_main(DCT_40_normalized, [1] * num_projections, DCT_40_train, model=projector, device=device, model_type='Memory', split=0.2, 
                                                     numProjections=num_projections, residual=True, loss_function=loss_function, model_dir='Model_Paths/DCT_40_proj15', display=True)

# Store Best Model Weights and History

save_progress(proj15_path, DCT_10_proj15_weights, DCT_10_proj15_hist, '10% Sampling')
save_progress(proj15_path, DCT_20_proj15_weights, DCT_20_proj15_hist, '20% Sampling')
save_progress(proj15_path, DCT_30_proj15_weights, DCT_30_proj15_hist, '30% Sampling')
save_progress(proj15_path, DCT_40_proj15_weights, DCT_40_proj15_hist, '40% Sampling')

Validation Size: 4500, Train Size: 18000
Epoch 1/300
Average training loss: 0.7528415042691761
Average test loss: 0.0053060034956369136
Epoch 2/300
Average training loss: 0.11119143628411823
Average test loss: 0.005061962634738949
Epoch 3/300
Average training loss: 0.08272468745708465
Average test loss: 0.004715821463200781
Epoch 4/300
Average training loss: 0.0758079924583435
Average test loss: 0.004594141910473506
Epoch 5/300
Average training loss: 0.0724109559059143
Average test loss: 0.004518310106876824
Epoch 6/300
Average training loss: 0.0702678271399604
Average test loss: 0.004503238973518213
Epoch 7/300
Average training loss: 0.06875890318221516
Average test loss: 0.0044075109888282085
Epoch 8/300
Average training loss: 0.06761713846524557
Average test loss: 0.004395750842988491
Epoch 9/300
Average training loss: 0.06666735280222363
Average test loss: 0.004436897427257564
Epoch 10/300
Average training loss: 0.06593333857589298
Average test loss: 0.004310242485668924
Epoch 11/3

In [10]:
DCT_10_proj15_model = MemoryNetwork(DCT_10_normalized, [1] * num_projections, num_projections, projector, device, residual=True).to(device)
DCT_20_proj15_model = MemoryNetwork(DCT_20_normalized, [1] * num_projections, num_projections, projector, device, residual=True).to(device)
DCT_30_proj15_model = MemoryNetwork(DCT_30_normalized, [1] * num_projections, num_projections, projector, device, residual=True).to(device)
DCT_40_proj15_model = MemoryNetwork(DCT_40_normalized, [1] * num_projections, num_projections, projector, device, residual=True).to(device)

DCT_10_proj15_psnr = average_PSNR(DCT_10_proj15_model, DCT_10_proj15_weights, num_projections, DCT_10_test, print_psnr=True)
DCT_20_proj15_psnr = average_PSNR(DCT_20_proj15_model, DCT_20_proj15_weights, num_projections, DCT_20_test, print_psnr=True)
DCT_30_proj15_psnr = average_PSNR(DCT_30_proj15_model, DCT_30_proj15_weights, num_projections, DCT_30_test, print_psnr=True)
DCT_40_proj15_psnr = average_PSNR(DCT_40_proj15_model, DCT_40_proj15_weights, num_projections, DCT_40_test, print_psnr=True)

with open(os.path.join(proj15_path, '10% Sampling', 'test_PSNR.pkl'), 'wb') as f:
    pickle.dump(DCT_10_proj15_psnr, f)
with open(os.path.join(proj15_path, '20% Sampling', 'test_PSNR.pkl'), 'wb') as f:
    pickle.dump(DCT_20_proj15_psnr, f)
with open(os.path.join(proj15_path, '30% Sampling', 'test_PSNR.pkl'), 'wb') as f:
    pickle.dump(DCT_30_proj15_psnr, f)
with open(os.path.join(proj15_path, '40% Sampling', 'test_PSNR.pkl'), 'wb') as f:
    pickle.dump(DCT_40_proj15_psnr, f)

Average PSNR for Projection Layer 0 across 2500 images: 27.44
Average PSNR for Projection Layer 1 across 2500 images: 27.58
Average PSNR for Projection Layer 2 across 2500 images: 27.83
Average PSNR for Projection Layer 3 across 2500 images: 27.93
Average PSNR for Projection Layer 4 across 2500 images: 27.92
Average PSNR for Projection Layer 5 across 2500 images: 28.16
Average PSNR for Projection Layer 6 across 2500 images: 28.07
Average PSNR for Projection Layer 7 across 2500 images: 28.20
Average PSNR for Projection Layer 8 across 2500 images: 28.23
Average PSNR for Projection Layer 9 across 2500 images: 28.24
Average PSNR for Projection Layer 10 across 2500 images: 28.26
Average PSNR for Projection Layer 11 across 2500 images: 28.27
Average PSNR for Projection Layer 12 across 2500 images: 28.30
Average PSNR for Projection Layer 13 across 2500 images: 28.31
Average PSNR for Projection Layer 14 across 2500 images: 28.30
Average PSNR for Projection Layer 0 across 2500 images: 27.84
Ave

In [11]:
# 30 Projections
num_projections = 30

# Folder Path for 30 Projections
proj30_path = 'Memory_Residual-DCT-Additive_Depth3-.01/30 Projections'

DCT_10_proj30_weights, DCT_10_proj30_hist = train_main(DCT_10_normalized, [1] * num_projections, DCT_10_train, model=projector, device=device, model_type='Memory', split=0.2, 
                                                     numProjections=num_projections, residual=True, loss_function=loss_function, model_dir='Model_Paths/DCT_10_proj30', display=True)

DCT_20_proj30_weights, DCT_20_proj30_hist = train_main(DCT_20_normalized, [1] * num_projections, DCT_20_train, model=projector, device=device, model_type='Memory', split=0.2, 
                                                     numProjections=num_projections, residual=True, loss_function=loss_function, model_dir='Model_Paths/DCT_20_proj30', display=True)

DCT_30_proj30_weights, DCT_30_proj30_hist = train_main(DCT_30_normalized, [1] * num_projections, DCT_30_train, model=projector, device=device, model_type='Memory', split=0.2, 
                                                     numProjections=num_projections, residual=True, loss_function=loss_function, model_dir='Model_Paths/DCT_30_proj30', display=True)

DCT_40_proj30_weights, DCT_40_proj30_hist = train_main(DCT_40_normalized, [1] * num_projections, DCT_40_train, model=projector, device=device, model_type='Memory', split=0.2, 
                                                     numProjections=num_projections, residual=True, loss_function=loss_function, model_dir='Model_Paths/DCT_40_proj30', display=True)

# Store Best Model Weights and History

save_progress(proj30_path, DCT_10_proj30_weights, DCT_10_proj30_hist, '10% Sampling')
save_progress(proj30_path, DCT_20_proj30_weights, DCT_20_proj30_hist, '20% Sampling')
save_progress(proj30_path, DCT_30_proj30_weights, DCT_30_proj30_hist, '30% Sampling')
save_progress(proj30_path, DCT_40_proj30_weights, DCT_40_proj30_hist, '40% Sampling')

Validation Size: 4500, Train Size: 18000
Epoch 1/300
Average training loss: 6.247368184195625
Average test loss: 0.00870816859851281
Epoch 2/300
Average training loss: 1.1503986932436625
Average test loss: 0.005020906614760558
Epoch 3/300
Average training loss: 0.5272008467780219
Average test loss: 0.004860191978100273
Epoch 4/300
Average training loss: 0.3551432976722717
Average test loss: 0.004887593905131022
Epoch 5/300
Average training loss: 0.2707812512980567
Average test loss: 0.004692467278904385
Epoch 6/300
Average training loss: 0.17103867714934878
Average test loss: 0.004461202621873882
Epoch 9/300
Average training loss: 0.16078011702166664
Average test loss: 0.004428372134764989
Epoch 10/300
Average training loss: 0.15369965783754985
Average test loss: 0.004415758913175927
Epoch 11/300
Average training loss: 0.14839806539482542
Average test loss: 0.0043920603278610445
Epoch 12/300
Average training loss: 0.14436287976635828
Average test loss: 0.004328922284973992
Epoch 13/300

In [12]:
DCT_10_proj30_model = MemoryNetwork(DCT_10_normalized, [1] * num_projections, num_projections, projector, device, residual=True).to(device)
DCT_20_proj30_model = MemoryNetwork(DCT_20_normalized, [1] * num_projections, num_projections, projector, device, residual=True).to(device)
DCT_30_proj30_model = MemoryNetwork(DCT_30_normalized, [1] * num_projections, num_projections, projector, device, residual=True).to(device)
DCT_40_proj30_model = MemoryNetwork(DCT_40_normalized, [1] * num_projections, num_projections, projector, device, residual=True).to(device)

DCT_10_proj30_psnr = average_PSNR(DCT_10_proj30_model, DCT_10_proj30_weights, num_projections, DCT_10_test, print_psnr=True)
DCT_20_proj30_psnr = average_PSNR(DCT_20_proj30_model, DCT_20_proj30_weights, num_projections, DCT_20_test, print_psnr=True)
DCT_30_proj30_psnr = average_PSNR(DCT_30_proj30_model, DCT_30_proj30_weights, num_projections, DCT_30_test, print_psnr=True)
DCT_40_proj30_psnr = average_PSNR(DCT_40_proj30_model, DCT_40_proj30_weights, num_projections, DCT_40_test, print_psnr=True)

with open(os.path.join(proj30_path, '10% Sampling', 'test_PSNR.pkl'), 'wb') as f:
    pickle.dump(DCT_10_proj30_psnr, f)
with open(os.path.join(proj30_path, '20% Sampling', 'test_PSNR.pkl'), 'wb') as f:
    pickle.dump(DCT_20_proj30_psnr, f)
with open(os.path.join(proj30_path, '30% Sampling', 'test_PSNR.pkl'), 'wb') as f:
    pickle.dump(DCT_30_proj30_psnr, f)
with open(os.path.join(proj30_path, '40% Sampling', 'test_PSNR.pkl'), 'wb') as f:
    pickle.dump(DCT_40_proj30_psnr, f)

Average PSNR for Projection Layer 0 across 2500 images: 27.23
Average PSNR for Projection Layer 1 across 2500 images: 27.50
Average PSNR for Projection Layer 2 across 2500 images: 27.61
Average PSNR for Projection Layer 3 across 2500 images: 27.68
Average PSNR for Projection Layer 4 across 2500 images: 27.63
Average PSNR for Projection Layer 5 across 2500 images: 27.95
Average PSNR for Projection Layer 6 across 2500 images: 28.00
Average PSNR for Projection Layer 7 across 2500 images: 27.84
Average PSNR for Projection Layer 8 across 2500 images: 28.06
Average PSNR for Projection Layer 9 across 2500 images: 28.10
Average PSNR for Projection Layer 10 across 2500 images: 28.08
Average PSNR for Projection Layer 11 across 2500 images: 28.02
Average PSNR for Projection Layer 12 across 2500 images: 28.09
Average PSNR for Projection Layer 13 across 2500 images: 28.04
Average PSNR for Projection Layer 14 across 2500 images: 28.12
Average PSNR for Projection Layer 15 across 2500 images: 28.17
Av